In [ ]:
!pip install pandas
!pip install pyspark
!pip install pyarrow

In [ ]:
from pyspark.sql import SparkSession
 
# Building the SparkSession and name 
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [3]:
import pandas as pd

data = [['1', '1', 'N'], ['2', '1', 'Y'], ['2', '2', 'N'], ['3', '3', 'N'], ['4', '2', 'N'], ['4', '3', 'Y'], ['4', '4', 'N']]
employee = pd.DataFrame(data, columns=['employee_id', 'department_id', 'primary_flag']).astype({'employee_id':'Int64', 'department_id':'Int64', 'primary_flag':'object'})

In [4]:
employee = spark.createDataFrame(employee)
employee.show()

+-----------+-------------+------------+
|employee_id|department_id|primary_flag|
+-----------+-------------+------------+
|          1|            1|           N|
|          2|            1|           Y|
|          2|            2|           N|
|          3|            3|           N|
|          4|            2|           N|
|          4|            3|           Y|
|          4|            4|           N|
+-----------+-------------+------------+



In [11]:
from pyspark.sql.functions import count

employee \
.where('primary_flag = "Y"') \
.select(['employee_id', 'department_id']) \
.union(
    employee \
    .groupby('employee_id') \
    .agg(count('department_id').alias('no_of_depts')) \
    .where('no_of_depts = 1') \
    .join(
        employee,
        'employee_id',
        'inner'
    ) \
    .select(['employee_id', 'department_id'])
) \
.show()


+-----------+-------------+
|employee_id|department_id|
+-----------+-------------+
|          2|            1|
|          4|            3|
|          1|            1|
|          3|            3|
+-----------+-------------+

